<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/isurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропуски отсутствуют. Все данные имеют числовой формат.

In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df['Пол'].unique()

array([1, 0], dtype=int64)

In [7]:
df['Члены семьи'].unique()

array([1, 0, 2, 4, 3, 5, 6], dtype=int64)

In [8]:
df['Возраст'].unique()

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

Отклонений в данных не обнаружено.

Проверим наличие дубликатов, однако в виду малого количества признаков и отсутствия столбца с полностью уникальными значениями вроде id или номера договора, не будем отбрасывать их.

In [9]:
df.duplicated().mean()

0.0306

Преобразовавать значения возраста и зарплаты и в формат цельных чисел int нет смысла, поскольку кодирование всё равно вернет их к float64.

## Умножение матриц

Необходимо ответить на вопрос:  
Изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу?

In [10]:
features = df.drop(columns = ('Страховые выплаты'))
target = df['Страховые выплаты']

Проверим методом кросс-валидации результаты линейной регрессии на исходных данных.

In [11]:
model = LinearRegression()
cross_val_score(model, features, target, scoring='r2', cv=4)

array([0.43045434, 0.42998266, 0.41239744, 0.42264271])

Теперь получим случайную квадратную матрицу пригодную для умножения признаков.  
Функцией np.linalg.inv проверим её обратимость(в случае необратимости получим сообщение об ошибке, вероятность этого крайне мала и в таком случае просто перезапустим ячейку).  

In [12]:
matrix = np.random.normal(size=(4, 4))
np.linalg.inv(matrix)
features = features.dot(matrix)

In [13]:
model = LinearRegression()
cross_val_score(model, features, target, scoring='r2', cv=4)

array([0.43045434, 0.42998266, 0.41239744, 0.42264271])

Результаты линейной регрессии остались **неизменными**.  
Рассмотрим формулы задействованные в вычислениях, чтобы объяснить это.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Матрица признаков после умножения будет равна - $XP$  
Соответственно формулы предсказания и обучения приобретут следующий вид: 

$$a_1 = XPw_1$$
$$w_1 = ((XP)^T XP)^{-1} (XP)^T y$$  

Пользуясь свойством ассоциативности($((XP)^T XP)^{-1}$ можно представить как $(((XP)^T X)P)^{-1}$) раскрываем первые скобки, применив одно из свойств обратной матрицы: $(AB)^{-1} = B^{-1}A^{-1}$:

$$w_1 = P^{-1}((XP)^T X)^{-1}(XP)^T y$$  

Далее раскрываем скобки транспонированных матриц:  

$$w_1 = P^{-1}(P^T X^T X)^{-1}P^T X^T y$$

Представив $(P^T X^T X)^{-1}$ как $(P^T (X^T X))^{-1}$, раскрываем скобки обратной матрицы:  

$$w_1 = P^{-1}(X^T X)^{-1}(P^T)^{-1}P^T X^T y$$

Поскольку $(P^T)^{-1}P^T = E$, а умножение на E ничего не меняет, сокращаем уравнение до:  

$$w_1 = P^{-1}(X^T X)^{-1}X^T y$$  

Поскольку $(X^T X)^{-1}X^T y = w$, получаем, что:

$$w_1 = P^{-1}w$$

Подставим это значение в формулу $a_1 = XPw_1$:  

$$a_1 = XPP^{-1}w$$  

Отбрасываем $PP^{-1}=E$ и получаем: $$a_1 = Xw = a$$

## Алгоритм преобразования

Наша задача состоит только в защите данных.  
То есть нужно привести данные к такому виду, чтобы не было возможности получить их в исходном виде, не имея доступа к самому алгоритму.  

In [15]:
def encode(feaures):
    try:
        coder = np.random.normal(size=(4, 4))
        coder = np.linalg.inv(coder)
        str_error = None
    except:
        coder = np.random.normal(size=(4, 4))
        coder = np.linalg.inv(coder)
    encoded_features = features.dot(coder)
    return encoded_features

Умножение на обратимую матрицу не только изменит признаки до неузнаваемости, но заменит названия столбцов.  
Использование обратной случайной матрицы выступает своебразной гарантией её обратимости.  
Конструкцию try/except применяется на случай генерации необратимой матрицы.

Осталось проверить алгоритм.  
Для начала еще раз взглянем на изначальные признаки.

In [16]:
features = df.drop(columns = ('Страховые выплаты'))
target = df['Страховые выплаты']
features.head(10)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
5,1,43.0,41000.0,2
6,1,39.0,39700.0,2
7,1,25.0,38600.0,4
8,1,36.0,49700.0,1
9,1,32.0,51700.0,1


In [17]:
model = LinearRegression()
cross_val_score(model, features, target, scoring='r2', cv=4)

array([0.43045434, 0.42998266, 0.41239744, 0.42264271])

Преобразуем их при помощи написанной функции.

In [18]:
features = encode(features)
features.head(10)

,0,1,2,3
0,-4243.192570,18876.715887,-12963.766733,2626.067106
1,-3239.027466,14451.646374,-9913.347409,1987.411903
2,-1787.357325,7984.410397,-5474.831432,1093.546956
3,-3577.749913,15878.499659,-10913.222431,2228.087551
4,-2227.835722,9929.104356,-6814.873257,1372.095504
5,-3499.587746,15596.443056,-10703.441806,2152.924269
6,-3390.718557,15103.700978,-10367.205981,2088.903253
7,-3306.238831,14692.473918,-10092.874716,2048.061774
8,-4255.844407,18918.304690,-12996.088360,2639.838302
9,-4431.518098,19683.420072,-13525.727402,2755.255916


Значения изменились до неузнаваемости, даже определить категорию столбца не прдставляется возможным.  
Проверим результаты линейной регрессии на преобразованных данных.

In [19]:
model = LinearRegression()
cross_val_score(model, features, target, scoring='r2', cv=4)

array([0.43045434, 0.42998266, 0.41239744, 0.42264271])

Результаты идентичны, задача выполнена!

## Общий вывод

Предоставленные данные в полном объеме соответствовали поставленной задаче и не требовали дополнительной предобработки.  
Был дан и обоснован ответ на вопрос об изменении качества линейной регрессии при умножении признаков на обратимую матрицу.  
Алгоритм преобразования данных был написан и успешно проверен.